In [ ]:
import torch
from torch_geometric.datasets import *
import torch.nn.functional as F
from torch_geometric.nn import GCNConv,GNNExplainer
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
dataset = KarateClub()
G = nx.karate_club_graph()
print(dataset[0])

In [ ]:
def plot_graph(label,G,title):
    '''
    G: a networkx G
    '''
    label = label.cpu().numpy()
    for i in range (34):
        (G.nodes[i]['club'])=label[i]
    
    plt.figure()
    plt.title('Epoch '+str(title))
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None    
    nodelist1 = []
    nodelist2 = []
    nodelist3 = []
    nodelist4 = []
    for i in range (34):
        if G.nodes[i]['club'] == 0:
            nodelist1.append(i)
        elif G.nodes[i]['club'] == 1:
            nodelist2.append(i)
        elif G.nodes[i]['club'] == 2:
            nodelist3.append(i)
        elif G.nodes[i]['club'] == 3:
            nodelist4.append(i)
                #nx.draw_networkx(G, pos, edges=edges);
    nx.draw(G,pos, with_labels=True)        
    nx.draw_networkx_nodes(G, pos, nodelist=nodelist1, node_size=700, node_color='r',alpha = 0.8)
    nx.draw_networkx_nodes(G, pos, nodelist=nodelist2, node_size=700, node_color='b',alpha = 0.8)
    nx.draw_networkx_nodes(G, pos, nodelist=nodelist3, node_size=700, node_color='y',alpha = 0.8)
    nx.draw_networkx_nodes(G, pos, nodelist=nodelist4, node_size=700, node_color='g',alpha = 0.8)                
    nx.draw_networkx_edges(G, pos, edgelist=edges,alpha =0.4)


In [1]:
import torch
from torch_geometric.datasets import *
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

dataset = KarateClub()

print(dataset[0])
print(dataset[0].y)


AttributeError: module 'scipy.sparse' has no attribute 'coo_array'

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
        pass

    def forward(self,x,edge_index):
        #x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)
    pass
pass

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
x, edge_index = data.x, data.edge_index

for epoch in range(301):
    
    optimizer.zero_grad()
    out = model(x,edge_index) #log_logits = model(x, edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask]) #loss = F.nll_loss(log_logits[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    #print('Epoch {} | Loss: {:.4f}'.format(epoch,loss.item()))
    model.eval()
    _, pred = model(x,edge_index).max(dim=1)
    #print(pred)
    #print(pred.eq(data.y))
    correct = int(pred.eq(data.y).sum().item())
    acc = correct / int(data.x.sum())
    #print('Accuracy:{:.4f}'.format(acc))
    print('Epoch {} | Loss: {:.4f}'.format(epoch,loss.item())+' | Accuracy:{:.4f}'.format(acc))
    if epoch%20==0:
        plot_graph(pred,G,epoch)
        plt.show()

    pass
pass

In [ ]:
explainer = GNNExplainer(model, epochs=300)
node_idx = 12
node_feat_mask, edge_mask = explainer.explain_node(node_idx, x, edge_index)
ax, G = explainer.visualize_subgraph(node_idx, edge_index, edge_mask, y=data.y)
plt.show()